<a href="https://colab.research.google.com/github/LuigiAjello/Programacao_linear/blob/main/1b2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# ==========================================================
# devlab 1b2 – Problema de Transporte com PuLP (Python)
# Objetivo: mandar produtos dos armazéns A, B, C para
# projetos 1, 2, 3 gastando o MENOR custo possível.
# ==========================================================

# Instalar a biblioteca, se necessário:
!pip install pulp

from pulp import *

# ----------------------------------------------------------
# 1) Dados de entrada
# ----------------------------------------------------------
# Armazéns (onde o produto está disponível)
warehouses = ["A", "B", "C"]

# Projetos (destino / clientes)
projetos = ["1", "2", "3"]

# Quanto cada armazém pode enviar
ofertas = {"A": 300,
           "B": 600,
           "C": 600}

# Quanto cada projeto precisa receber
demandas = {"1": 150,
            "2": 450,
            "3": 900}

# Custo para levar UMA unidade de cada armazém a cada projeto
custos = [
    [5, 1, 9],   # custos de A → (1, 2, 3)
    [4, 2, 8],   # custos de B → (1, 2, 3)
    [8, 7, 2]    # custos de C → (1, 2, 3)
]

# Converte a lista de custos em dicionário compreendido pelo PuLP
matriz_custos = makeDict([warehouses, projetos], custos, 0)

# ----------------------------------------------------------
# 2) Construção do modelo
# ----------------------------------------------------------
model = LpProblem("Problema_de_transporte", LpMinimize)

# Variáveis de decisão: fluxo de produto em cada rota (w, p)
variaveis = LpVariable.dicts(
    "Fluxo",           # prefixo do nome
    (warehouses, projetos),
    lowBound=0,        # não pode ser negativo
    cat="Continuous"   # pode ser fracionário; troque para "Integer" se quiser inteiros
)

# ----------------------------------------------------------
# 3) Função-objetivo (minimizar custo total)
# ----------------------------------------------------------
rotas = [(w, p) for w in warehouses for p in projetos]
model += lpSum(variaveis[w][p] * matriz_custos[w][p] for (w, p) in rotas)

# ----------------------------------------------------------
# 4) Restrições
# ----------------------------------------------------------
# (a) Cada armazém envia no máximo o que tem
for w in warehouses:
    model += lpSum(variaveis[w][p] for p in projetos) <= ofertas[w], \
             f"Limite_oferta_{w}"

# (b) Cada projeto recebe pelo menos o que precisa
for p in projetos:
    model += lpSum(variaveis[w][p] for w in warehouses) >= demandas[p], \
             f"Atende_demanda_{p}"

# ----------------------------------------------------------
# 5) Resolver e exibir resultados
# ----------------------------------------------------------
model.solve()

print("Status:", LpStatus[model.status])
for v in model.variables():
    print(f"{v.name} = {v.varValue}")

if model.status == 1:  # 1 = solução ótima encontrada
    print("Custo total mínimo:", value(model.objective))
else:
    print("O modelo não foi resolvido com sucesso.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 98.7 MB/s eta 0:00:00
Status: Optimal
Fluxo_A_1 = 0.0
Fluxo_A_2 = 300.0
Fluxo_A_3 = 0.0
Fluxo_B_1 = 150.0
Fluxo_B_2 = 150.0
Fluxo_B_3 = 300.0
Fluxo_C_1 = 0.0
Fluxo_C_2 = 0.0
Fluxo_C_3 = 600.0
Custo total mínimo: 4800.0
